# Image to Text

In [1]:
from devtools import debug
from dotenv import load_dotenv

load_dotenv()

!export PYTHONPATH=":./python"

In [5]:
from python.ai_core.llm import get_llm
from python.ai_core.prompts import def_prompt

#llm = get_llm(llm_id="gpt_4o_azure")  # Select an LLM that supports Vision
llm = get_llm(llm_id="gpt_4o_openai")  # Select an LLM that supports Vision

2024-07-03 23:58:24.173 | INFO     | python.ai_core.llm:get_llm:405 - get LLM : gpt_4o_azure - configurable: True
2024-07-03 23:58:24.861 | INFO     | python.ai_core.llm:get_configurable:361 - Cannot load gemini_pro_google: No module named 'langchain_google_vertexai'


In [6]:
import base64

IMAGE_PATH = "use_case_data/railway/network rail.png"


# Open the image file and encode it as a base64 string
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")


base64_image = encode_image(IMAGE_PATH)

messages = [
    {
        "role": "system",
        "content": "You are a helpful assistant that responds in Markdown.",
    },
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "Describe the images as an alternative text"},
            {
                "type": "image_url",
                "image_url": {"url": f"data:image/png;base64,{base64_image}"},
            },
        ],
    },
]

r = llm.invoke(messages)
print(r)

NotFoundError: Error code: 404 - {'error': {'code': 'DeploymentNotFound', 'message': 'The API deployment for this resource does not exist. If you created the deployment within the last 5 minutes, please wait a moment and try again.'}}